In [1]:
%run functions.py
%run models.py

/Users/dayn/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from datasets import load_dataset

ds = load_dataset("goendalf666/sales-conversations-instruction-customer")

In [3]:
full_ds = keep_full_convos(ds)

In [4]:
train_set, test_set = split_train_test(full_ds)

In [5]:
train_set[:5]

['Customer: Can you please explain to me how this product works? Salesman: Certainly! This product is designed to simplify your workflow by automating repetitive tasks. It uses advanced algorithms to analyze data and generate actionable insights, saving you time and effort. Customer: Im not sure if this service is worth the price. Can you explain its benefits to me? Salesman: Absolutely! This service offers numerous benefits, such as increased productivity, improved efficiency, and cost savings. It integrates seamlessly with your existing systems and provides real-time analytics, allowing you to make data-driven decisions and stay ahead of the competition. Customer: I have a limited technical background. Can you explain this feature in simpler terms? Salesman: Of course! Let me break it down for you. This feature essentially streamlines your customer support process by automating ticket management and routing. It ensures that customer inquiries are efficiently handled and assigned to t

In [6]:
test = train_set[0]

In [7]:
dict_test = split_cust_sales(test)

In [8]:
full_text_cust, full_text_sales = dict_test['full']
split_text_cust, split_text_sales = dict_test['split']

In [9]:
print("Before:", full_text_cust)
print("-------")
print("After:", summarise(full_text_cust))

Before: Customer says to salesman: Certainly! This product is designed to simplify your workflow by automating repetitive tasks. It uses advanced algorithms to analyze data and generate actionable insights, saving you time and effort.
Absolutely! This service offers numerous benefits, such as increased productivity, improved efficiency, and cost savings. It integrates seamlessly with your existing systems and provides real-time analytics, allowing you to make data-driven decisions and stay ahead of the competition.
Of course! Let me break it down for you. This feature essentially streamlines your customer support process by automating ticket management and routing. It ensures that customer inquiries are efficiently handled and assigned to the most appropriate team member, resulting in faster response times and improved customer satisfaction.
Absolutely! This product offers time savings, increased efficiency, and improved decision-making through advanced analytics. It simplifies your pr

In [10]:
split_text_cust

['Certainly! This product is designed to simplify your workflow by automating repetitive tasks. It uses advanced algorithms to analyze data and generate actionable insights, saving you time and effort.',
 'Absolutely! This service offers numerous benefits, such as increased productivity, improved efficiency, and cost savings. It integrates seamlessly with your existing systems and provides real-time analytics, allowing you to make data-driven decisions and stay ahead of the competition.',
 'Of course! Let me break it down for you. This feature essentially streamlines your customer support process by automating ticket management and routing. It ensures that customer inquiries are efficiently handled and assigned to the most appropriate team member, resulting in faster response times and improved customer satisfaction.',
 'Absolutely! This product offers time savings, increased efficiency, and improved decision-making through advanced analytics. It simplifies your processes and helps you

In [11]:
for line in split_text_cust:
    print(f"{line} | Prediction: {analyze_sentiment(line)}")

Certainly! This product is designed to simplify your workflow by automating repetitive tasks. It uses advanced algorithms to analyze data and generate actionable insights, saving you time and effort. | Prediction: neutral
Absolutely! This service offers numerous benefits, such as increased productivity, improved efficiency, and cost savings. It integrates seamlessly with your existing systems and provides real-time analytics, allowing you to make data-driven decisions and stay ahead of the competition. | Prediction: neutral
Of course! Let me break it down for you. This feature essentially streamlines your customer support process by automating ticket management and routing. It ensures that customer inquiries are efficiently handled and assigned to the most appropriate team member, resulting in faster response times and improved customer satisfaction. | Prediction: neutral
Absolutely! This product offers time savings, increased efficiency, and improved decision-making through advanced a

In [12]:
extract_keywords(full_text_cust)

'Automated repetitive tasks, advanced algorithms, data analysis, productivity, efficiency, cost savings, real-time analytics, data-driven decisions, customer support process, ticket management, customer inquiries, faster response times, customer satisfaction,'

In [13]:
train_set[:5]

['Customer: Can you please explain to me how this product works? Salesman: Certainly! This product is designed to simplify your workflow by automating repetitive tasks. It uses advanced algorithms to analyze data and generate actionable insights, saving you time and effort. Customer: Im not sure if this service is worth the price. Can you explain its benefits to me? Salesman: Absolutely! This service offers numerous benefits, such as increased productivity, improved efficiency, and cost savings. It integrates seamlessly with your existing systems and provides real-time analytics, allowing you to make data-driven decisions and stay ahead of the competition. Customer: I have a limited technical background. Can you explain this feature in simpler terms? Salesman: Of course! Let me break it down for you. This feature essentially streamlines your customer support process by automating ticket management and routing. It ensures that customer inquiries are efficiently handled and assigned to t

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Sample dataset of conversations
data = train_set

# Preprocessing function
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    
    # Join tokens back into a string
    processed_text = ' '.join(tokens)
    
    return processed_text

# Preprocess each conversation
processed_data = [preprocess_text(conv) for conv in data]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_data)

# Clustering with K-means
num_clusters = 3  # Number of clusters (you can adjust this)
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(X)

# Assign clusters to original data
clustered_data = pd.DataFrame({'Conversation': data, 'Cluster': clusters})

# Display the clustered data
print(clustered_data)


/Users/dayn/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


                                           Conversation  Cluster
0     Customer: Can you please explain to me how thi...        1
1     Customer: Im interested in your product, but I...        1
2     Customer: Hi, Im interested in learning more a...        1
3     Customer: Im interested in this new health pro...        2
4     Customer: Im looking for a financial product t...        0
...                                                 ...      ...
2723  Customer: Im interested in your financial prod...        0
2724  Customer: Im interested in your health supplem...        2
2725  Customer: Hi, Im interested in your health pro...        2
2726  Customer: Hi, Im looking for a new health supp...        2
2727  Customer: Im interested in your product, but I...        1

[2728 rows x 2 columns]


In [15]:
for i in range(10,20):
    print(clustered_data[clustered_data['Cluster'] == 1].iloc[i]['Conversation'])

Customer: Im currently facing some challenges with my financial operations. Can you tell me more about how you can help? Salesman: Of course! Id be happy to. Can you provide some specific examples of the challenges youre facing? Customer: What are your goals for the coming year? Salesman: Thats a great question. Id love to hear more about what youre looking to achieve. Can you share some of your goals with me? Customer: Ive been experiencing difficulties with scalability in my current system. Can you help me with that? Salesman: Absolutely. Just to clarify, are you saying that the lack of scalability is your main pain point? Customer: Im not sure if your solution can address my specific needs. Can you explain how it can help? Salesman: Certainly. Lets connect your needs to our solution. Can you tell me more about your pain points so I can show you how our product can address them? Customer: I have multiple needs and pain points. How do you suggest we prioritize them? Salesman: Thats a 

In [16]:
import random
import faker

fake = faker.Faker()

# Mocking client records
def mock_clients(num_clients):
    clients = []
    for i in range(num_clients):
        client_id = i + 1
        client_name = fake.company()
        clients.append((client_id, client_name))
    return clients
    # df_clients = pd.DataFrame(clients, columns=['id', 'name'])
    # return df_clients

# Mocking contact records
def mock_contacts(num_contacts, clients):
    contacts = []
    for i in range(num_contacts):
        contact_id = i + 1
        client_id = random.choice(clients)[0]
        contact_name = fake.name()
        phone_number = fake.phone_number()
        contacts.append((contact_id, client_id, contact_name, phone_number))
    return contacts
    # df_contacts = pd.DataFrame(contacts, columns=['id', 'client_id', 'name', 'phone_number'])
    # return df_contacts

# Mocking call records
def mock_call_records(num_calls, clients, contacts):
    call_records = []
    for i in range(num_calls):
        call_id = i + 1
        client_id = random.choice(clients)[0]
        contact_id = random.choice(contacts)[0]
        phone_number = fake.phone_number()
        call_records.append((call_id, client_id, contact_id, phone_number))
    return call_records
    # df_call_records = pd.DataFrame(call_records, columns=['id', 'client_id', 'contact_id', 'phone_number'])
    # return df_call_records

# Example usage
num_clients = 5
num_contacts = 10
num_calls = 20

clients = mock_clients(num_clients)
contacts = mock_contacts(num_contacts, clients)
call_records = mock_call_records(num_calls, clients, contacts)

In [17]:
df_clients = pd.DataFrame(clients, columns=['id', 'name'])
df_contacts = pd.DataFrame(contacts, columns=['id', 'client_id', 'name', 'phone_number'])
df_callrecords = pd.DataFrame(call_records, columns=['id', 'client_id', 'contact_id', 'phone_number'])

In [18]:
%run mock_conversations.py

In [19]:
convos['convo5']

[{'speaker': 'TikTok Sales Representative (Emily)',
  'role': 'salesman',
  'message': "Hi there! I'm Emily from TikTok's advertising team. How are you today?"},
 {'speaker': 'Potential Client (Jessica)',
  'role': 'customer',
  'message': "Hi Emily, I'm doing well, thank you. How about you?"},
 {'speaker': 'Emily',
  'role': 'salesman',
  'message': "I'm great, thanks! I'm excited to discuss how TikTok's advertising solutions can benefit your business. Could you tell me a bit about your business and your advertising goals?"},
 {'speaker': 'Jessica',
  'role': 'customer',
  'message': 'Sure! We run a small boutique specializing in handmade jewelry. Our goal is to increase online sales and build brand awareness among a younger audience.'},
 {'speaker': 'Emily',
  'role': 'salesman',
  'message': "That's fantastic. TikTok could be a great platform to showcase your unique products through engaging video content. Have you considered advertising on TikTok before?"},
 {'speaker': 'Jessica',


In [20]:
def extract(convo_ds):
    cust, sales = [], []
    for entry in convo_ds:
        if entry['role'] == 'customer':
            cust.append(entry['message'])
        if entry['role'] == 'salesman':
            sales.append(entry['message'])
    return cust, sales

list_cust, list_sales = extract(convos['convo5'])
full_cust, full_sales = ' '.join(list_cust), ' '.join(list_sales)

In [21]:
summarise(full_cust)

'Emily advises a small boutique on how to increase online sales and build brand awareness among a younger audience. The boutique is in the early stages of planning its marketing strategy for the next quarter and needs more time to assess other social media platforms.'

In [22]:
extract_keywords(full_cust)

'marketing strategy, marketing plans, budget, online sales, brand awareness, social media platforms'

In [23]:
for line in list_cust:
    print(f"{line} | Prediction: {analyze_sentiment(line)}")

Hi Emily, I'm doing well, thank you. How about you? | Prediction: joy
Sure! We run a small boutique specializing in handmade jewelry. Our goal is to increase online sales and build brand awareness among a younger audience. | Prediction: joy
We've thought about it, but we're also exploring other social media platforms. We're looking for the most cost-effective solution that aligns with our budget. | Prediction: neutral
Not exactly. We're still in the early stages of planning our marketing strategy for the next quarter. | Prediction: neutral
I appreciate the information, Emily. However, I think we'll need more time to evaluate our options and compare them with other platforms before making a decision. | Prediction: joy
That sounds good to me. I should have a clearer picture of our marketing plans by then. | Prediction: neutral
Thank you, Emily. I appreciate your understanding and assistance. | Prediction: joy
You too, Emily. Take care! | Prediction: anger


In [24]:
for line in list_sales:
    print(f"{line} | Prediction: {analyze_sentiment(line)}")

Hi there! I'm Emily from TikTok's advertising team. How are you today? | Prediction: neutral
I'm great, thanks! I'm excited to discuss how TikTok's advertising solutions can benefit your business. Could you tell me a bit about your business and your advertising goals? | Prediction: joy
That's fantastic. TikTok could be a great platform to showcase your unique products through engaging video content. Have you considered advertising on TikTok before? | Prediction: joy
I understand. TikTok offers various ad formats and targeting options that can be tailored to fit different budgets. Have you determined what kind of budget you'd be comfortable with for advertising? | Prediction: neutral
No problem at all. Would you be interested in hearing more about our ad solutions and how they could potentially benefit your business? | Prediction: neutral
I completely understand. It's important to explore all your options thoroughly. Would it be okay if I follow up with you in a few weeks to see if you'

In [25]:
# Define follow-up action keywords
follow_up_keywords = ["schedule", "follow up", "set up", "arrange", "send", "discuss", "plan"]

# Function to extract follow-up actions from a conversation
def extract_follow_up_actions(convo):
    follow_up_actions = []
    for message in convo:
        for keyword in follow_up_keywords:
            if keyword in message["message"].lower():
                follow_up_actions.append({
                    "speaker": message["speaker"],
                    "role": message["role"],
                    "message": message["message"]
                })
                break  # Stop checking further keywords once a match is found
    return follow_up_actions

# Extract follow-up actions from each conversation in your convos dataset
all_follow_up_actions = {}
for convo_name, conversation in convos.items():
    follow_up_actions = extract_follow_up_actions(conversation)
    all_follow_up_actions[convo_name] = follow_up_actions

# Print the extracted follow-up actions for each conversation
for convo_name, actions in all_follow_up_actions.items():
    print(f"Conversation '{convo_name}':")
    for action in actions:
        print(f"- {action['role']} ({action['speaker']}): {action['message']}")
    print()


Conversation 'convo1':
- salesman (Alex): I'm great, thank you! I'm excited to discuss how TikTok can help your brand reach a wider audience and drive engagement. Could you tell me a bit about your business and your current marketing strategies?
- salesman (Alex): We can set up a follow-up meeting with our creative and marketing teams to discuss a customized strategy for your brand. We'll also provide you with a detailed proposal outlining our recommendations and potential costs.
- customer (Jamie): Sounds good, Alex. Let's schedule that follow-up meeting.
- salesman (Alex): Fantastic! I'll send you an email with available times for next week. Thank you for your time, Jamie. I look forward to working together to make your brand shine on TikTok!

Conversation 'convo2':
- salesman (Sarah): Costs can vary based on the influencer's reach and engagement rates. We can discuss budget options and find influencers who fit within your marketing budget while delivering impactful results.
- salesm

In [26]:
import re

# Define follow-up action keywords
follow_up_keywords = ["schedule", "follow up", "set up", "arrange", "send", "discuss", "plan"]

# Define patterns for timeframe extraction
timeframe_patterns = [
    r"(\bnext\b\s+\w+)",  # Matches phrases like "next week", "next month"
    r"(\btomorrow\b|\btonight\b)",  # Matches specific days like "tomorrow" or "tonight"
    r"(\b\d{1,2}\/\d{1,2}\/\d{4}\b)",  # Matches dates in format dd/mm/yyyy
    r"(\b\d{1,2}-\d{1,2}-\d{4}\b)",  # Matches dates in format dd-mm-yyyy
    r"(\b\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{4}\b)",  # Matches dates like "1 Jan 2024"
    r"(\b(?:\w+\s+)?(?:weeks?|days?)\b)"
]

# Function to extract follow-up actions and timeframes from a conversation
def extract_follow_up_actions_with_timeframes(convo):
    follow_up_actions = []
    for message in convo:
        message_text = message["message"].lower()
        found_action = False
        for keyword in follow_up_keywords:
            if keyword in message_text:
                found_action = True
                follow_up_actions.append({
                    "speaker": message["speaker"],
                    "role": message["role"],
                    "message": message["message"],
                    "timeframe": None  # Placeholder for timeframe
                })
                break  # Stop checking further keywords once a match is found
        
        if found_action:
            # Extract timeframe using patterns
            for pattern in timeframe_patterns:
                match = re.search(pattern, message_text)
                if match:
                    follow_up_actions[-1]["timeframe"] = match.group().strip()
                    break  # Stop searching further patterns once a match is found

    return follow_up_actions

# Example: Extract follow-up actions with timeframes from each conversation in your convos dataset
all_follow_up_actions_with_timeframes = {}
for convo_name, conversation in convos.items():
    follow_up_actions = extract_follow_up_actions_with_timeframes(conversation)
    all_follow_up_actions_with_timeframes[convo_name] = follow_up_actions

# Print the extracted follow-up actions with timeframes for each conversation
for convo_name, actions in all_follow_up_actions_with_timeframes.items():
    print(f"Conversation '{convo_name}':")
    for action in actions:
        print(f"- {action['role']} ({action['speaker']}): {action['message']} | Timeframe: {action['timeframe']}")
    print()


Conversation 'convo1':
- salesman (Alex): I'm great, thank you! I'm excited to discuss how TikTok can help your brand reach a wider audience and drive engagement. Could you tell me a bit about your business and your current marketing strategies? | Timeframe: None
- salesman (Alex): We can set up a follow-up meeting with our creative and marketing teams to discuss a customized strategy for your brand. We'll also provide you with a detailed proposal outlining our recommendations and potential costs. | Timeframe: None
- customer (Jamie): Sounds good, Alex. Let's schedule that follow-up meeting. | Timeframe: None
- salesman (Alex): Fantastic! I'll send you an email with available times for next week. Thank you for your time, Jamie. I look forward to working together to make your brand shine on TikTok! | Timeframe: next week

Conversation 'convo2':
- salesman (Sarah): Costs can vary based on the influencer's reach and engagement rates. We can discuss budget options and find influencers who 

In [34]:
# Example of integrating semantic compatibility scoring with existing approach

from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Fine-tuned BERT model and tokenizer (replace with your fine-tuned model)
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


def find_best_action_noun_pair(actions, nouns):
    max_score = -float('inf')
    best_action_noun_pair = None
    
    for action in actions:
        for noun in nouns:
            # Prepare inputs for BERT
            input_text = f"{action} {noun}"
            inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

            # Forward pass through BERT
            with torch.no_grad():
                outputs = model(**inputs)

            # Extract logits and softmax to get probabilities
            probabilities = torch.softmax(outputs.logits, dim=-1)[0].numpy()

            # Assuming index 1 is the positive class score
            score = probabilities[1]

            # Update best pair if score is higher
            if score > max_score:
                max_score = score
                best_action_noun_pair = (action, noun)
    
    return best_action_noun_pair

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
import spacy

# Load the SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Define follow-up keywords
follow_up_keywords = ["schedule", "follow up", "set up", "arrange", "send", "discuss", "plan"]

# Function to extract follow-up actions with nouns using SpaCy
def extract_follow_up_actions_with_nouns(convo):
    follow_up_actions = []
    for message in convo:
        doc = nlp(message["message"])
        found_action = False
        for keyword in follow_up_keywords:
            if keyword in message["message"].lower():
                found_action = True
                action_nouns = []
                for token in doc:
                    if token.dep_ == "dobj":  # Check if token is a direct object
                        action_nouns.append(token.text)
                follow_up_actions.append({
                    "speaker": message["speaker"],
                    "role": message["role"],
                    "message": message["message"],
                    "action": keyword,
                    "nouns": action_nouns,
                    "timeframe": None  # Placeholder for timeframe

                })
                break  # Stop checking further keywords once a match is found

        if found_action:
            # Extract temporal entities (e.g., dates, times, durations) from the processed text
            for ent in doc.ents:
                if ent.label_ in ["DATE", "TIME", "DURATION"]:
                    follow_up_actions[-1]["timeframe"] = ent.text
                    break  # Take the first temporal entity found

    return follow_up_actions
# Example: Extract follow-up actions with nouns from each conversation in your convos dataset
all_follow_up_actions_with_nouns = {}
for convo_name, conversation in convos.items():
    follow_up_actions = extract_follow_up_actions_with_nouns(conversation)
    all_follow_up_actions_with_nouns[convo_name] = follow_up_actions

# Print the extracted follow-up actions with nouns for each conversation
for convo_name, actions in all_follow_up_actions_with_nouns.items():
    print(f"Conversation '{convo_name}':")
    for action in actions:
        print(f"- {action['role']} ({action['speaker']}): {action['message']} | Action: {find_best_action_noun_pair([action['action']], action['nouns'])} | Timeframe: {action['timeframe']}")
    print()


Conversation 'convo1':
- salesman (Alex): I'm great, thank you! I'm excited to discuss how TikTok can help your brand reach a wider audience and drive engagement. Could you tell me a bit about your business and your current marketing strategies? | Action: ('discuss', 'bit') | Timeframe: None
- salesman (Alex): We can set up a follow-up meeting with our creative and marketing teams to discuss a customized strategy for your brand. We'll also provide you with a detailed proposal outlining our recommendations and potential costs. | Action: ('set up', 'strategy') | Timeframe: None
- customer (Jamie): Sounds good, Alex. Let's schedule that follow-up meeting. | Action: None | Timeframe: None
- salesman (Alex): Fantastic! I'll send you an email with available times for next week. Thank you for your time, Jamie. I look forward to working together to make your brand shine on TikTok! | Action: ('send', 'email') | Timeframe: next week

Conversation 'convo2':
- salesman (Sarah): Costs can vary base

In [36]:
for convo_name, actions in all_follow_up_actions_with_nouns.items():
    print(actions)

[{'speaker': 'Alex', 'role': 'salesman', 'message': "I'm great, thank you! I'm excited to discuss how TikTok can help your brand reach a wider audience and drive engagement. Could you tell me a bit about your business and your current marketing strategies?", 'action': 'discuss', 'nouns': ['you', 'engagement', 'bit'], 'timeframe': None}, {'speaker': 'Alex', 'role': 'salesman', 'message': "We can set up a follow-up meeting with our creative and marketing teams to discuss a customized strategy for your brand. We'll also provide you with a detailed proposal outlining our recommendations and potential costs.", 'action': 'set up', 'nouns': ['meeting', 'strategy', 'you', 'recommendations'], 'timeframe': None}, {'speaker': 'Jamie', 'role': 'customer', 'message': "Sounds good, Alex. Let's schedule that follow-up meeting.", 'action': 'schedule', 'nouns': [], 'timeframe': None}, {'speaker': 'Alex', 'role': 'salesman', 'message': "Fantastic! I'll send you an email with available times for next wee